In [1]:
import pandas as pd
import os
import warnings 

warnings.filterwarnings('ignore')

# 14.1 指标波动贡献率

## 14.1.1 什么是贡献率

## 14.1.2 可加型指标波动贡献率的计算

### 1. 计算逻辑（书中有具体案例讲解）

### 2.可加型指标波动贡献 的Pandas实现

#### 构造上面案例数据集

In [3]:
d1 = pd.DataFrame({'渠道':['A','B','C'],
                   'before':[11000,500,300],
                   'after':[12000,1500,800]})
d1

,渠道,before,after
0,A,11000,12000
1,B,500,1500
2,C,300,800


#### 计算环比、波动值、波动贡献率

In [4]:
d1['环比'] = (d1['after'] - d1['before']) / d1['before']
d1['波动值'] = d1['after'] - d1['before']
d1['波动贡献率'] = d1['波动值'] / d1['波动值'].sum()     #用每一行波动值除以总体波动值汇总得到贡献度

d1

,渠道,before,after,环比,波动值,波动贡献率
0,A,11000,12000,0.090909,1000,0.4
1,B,500,1500,2.000000,1000,0.4
2,C,300,800,1.666667,500,0.2


## 14.1.3 乘法型指标波动贡献率的计算

### 1.计算逻辑（书中有具体案例讲解）

### 2.乘法型指标波动贡献的Pandas实现

#### 构造数据

In [5]:
d2 = pd.DataFrame({'指标':['访客数','转化率','客单价','销售额'],
                   'before':[10000,0.05,350,175000],
                   'after':[15000,0.08,330,396000]})

d2

,指标,before,after
0,访客数,10000.00,15000.00
1,转化率,0.05,0.08
2,客单价,350.00,330.00
3,销售额,175000.00,396000.00


#### 计算环比、LN后的值和波动贡献率

In [7]:
import numpy as np #numpy的log可以直接计算一个数组

d2['环比'] = (d2['after'] - d2['before']) / d2['before']
d2['LN_before'] = np.log(d2['before']) 
d2['LN_after'] = np.log(d2['after'])
d2['LN波动值'] =  d2['LN_after'] - d2['LN_before']
d2['波动贡献率'] = d2['LN波动值'] / d2['LN波动值'][3]   #总体数据索引是3，因此这里用3来找到总体值

d2

,指标,before,after,环比,LN_before,LN_after,LN波动值,波动贡献率
0,访客数,10000.00,15000.00,0.500000,9.210340,9.615805,0.405465,0.496511
1,转化率,0.05,0.08,0.600000,-2.995732,-2.525729,0.470004,0.575542
2,客单价,350.00,330.00,-0.057143,5.857933,5.799093,-0.058841,-0.072053
3,销售额,175000.00,396000.00,1.262857,12.072541,12.889169,0.816628,1.000000


In [8]:
d2['LN波动值']

0    0.405465
1    0.470004
2   -0.058841
3    0.816628
Name: LN波动值, dtype: float64

In [9]:
d2['LN波动值'][3]

0.816628237330967

## 14.1.4 除法型指标波动贡献率的计算

### 1.计算逻辑（书中有具体案例讲解）

### 2. 波动贡献率剖析

### 3. 除法型指标波动贡献的Pandas实现

#### 构造数据

In [10]:
#访客数
df_uv = pd.DataFrame({
    'channel':['A','B','C','D','ALL'],
    'before_uv':[50000,10000,30000,10000,100000],
    'after_uv':[100000,10000,50000,25000,185000]
})

#购买人数
df_pay = pd.DataFrame({
    'channel':['A','B','C','D','ALL'],
    'before_pay':[10000,1500,15000,1000,27500],
    'after_pay':[20000,1500,26000,1000,48500]
})

#转化率
df_cvr = pd.DataFrame({
    'channel':['A','B','C','D','ALL'],
    'before_cvr':[0.2,0.15,0.5,0.1,0.28],
    'after_cvr':[0.2,0.15,0.52,0.04,0.26]
})

In [11]:
df_uv

,channel,before_uv,after_uv
0,A,50000,100000
1,B,10000,10000
2,C,30000,50000
3,D,10000,25000
4,ALL,100000,185000


In [12]:
df_pay

,channel,before_pay,after_pay
0,A,10000,20000
1,B,1500,1500
2,C,15000,26000
3,D,1000,1000
4,ALL,27500,48500


In [13]:
df_cvr

,channel,before_cvr,after_cvr
0,A,0.20,0.20
1,B,0.15,0.15
2,C,0.50,0.52
3,D,0.10,0.04
4,ALL,0.28,0.26


#### 构造一张活动前后数据均不变的表，为后续的控制变量做准备

In [7]:
df_uv_con = df_uv.copy()
df_uv_con['after_uv'] = df_uv['before_uv']

df_pay_con = df_pay.copy()
df_pay_con['after_pay'] = df_pay['before_pay']

In [8]:
df_uv_con

,channel,before_uv,after_uv
0,A,50000,50000
1,B,10000,10000
2,C,30000,30000
3,D,10000,10000
4,ALL,100000,100000


In [9]:
df_pay_con

,channel,before_pay,after_pay
0,A,10000,10000
1,B,1500,1500
2,C,15000,15000
3,D,1000,1000
4,ALL,27500,27500


#### 控制变量的方式计算A渠道的影响，即其他渠道活动前后数据不变，计算A渠道数据的变化对整体的影响

In [10]:
#所有渠道不变，只有A渠道是真实的值，访客和购买人数均如此
df_uv_con.loc[df_uv_con['channel'] == 'A',:] = df_uv.loc[df_uv['channel'] == 'A',:]
df_pay_con.loc[df_pay_con['channel'] == 'A',:] = df_pay.loc[df_pay['channel'] == 'A',:]

In [11]:
df_uv_con

,channel,before_uv,after_uv
0,A,50000,100000
1,B,10000,10000
2,C,30000,30000
3,D,10000,10000
4,ALL,100000,100000


In [12]:
df_pay_con

,channel,before_pay,after_pay
0,A,10000,20000
1,B,1500,1500
2,C,15000,15000
3,D,1000,1000
4,ALL,27500,27500


#### 重新计算汇总值，并计算A渠道的影响

In [13]:
#对活动前后的访客数进行汇总并重新计算
before_uv_sum = df_uv_con['before_uv'][:-1].sum()
after_uv_sum = df_uv_con['after_uv'][:-1].sum()

#对活动前后的购买人数进行汇总并重新计算
before_pay_sum = df_pay_con['before_pay'][:-1].sum()
after_pay_sum = df_pay_con['after_pay'][:-1].sum()

#计算A渠道对整体的影响，活动后整体转化率和活动前的环比
before_cvr_all = before_pay_sum / before_uv_sum
after_cvr_all = after_pay_sum / after_uv_sum
result_a = (after_cvr_all - before_cvr_all) / before_cvr_all

#显示4位小数的结果，避免小数位太多
print(round(result_a,4))

-0.0909


#### 其他渠道也是同样的控制变量逻辑

In [14]:
result_dic = {}

for channel in df_uv['channel'][:-1]:
    
    #构造控制变量的原始表，活动前后数据相等
    df_uv_con = df_uv.copy()
    df_uv_con['after_uv'] = df_uv['before_uv']

    df_pay_con = df_pay.copy()
    df_pay_con['after_pay'] = df_pay['before_pay']
    
    #用索引替换对应渠道的数据
    df_uv_con.loc[df_uv_con['channel'] == channel,:] = df_uv.loc[df_uv['channel'] == channel,:]
    df_pay_con.loc[df_pay_con['channel'] == channel,:] = df_pay.loc[df_pay['channel'] == channel,:]
    
    
    #对活动前后的访客数进行汇总并重新计算
    before_uv_sum = df_uv_con['before_uv'][:-1].sum()
    after_uv_sum = df_uv_con['after_uv'][:-1].sum()

    #对活动前后额购买人数进行汇总并重新计算
    before_pay_sum = df_pay_con['before_pay'][:-1].sum()
    after_pay_sum = df_pay_con['after_pay'][:-1].sum()

    #计算渠道对整体的影响，活动后整体转化率和活动前的环比
    before_cvr_all = before_pay_sum / before_uv_sum
    after_cvr_all = after_pay_sum / after_uv_sum
    result_x = (after_cvr_all - before_cvr_all) / before_cvr_all

    #显示两位小数的结果，避免小数位太多
    result_dic[channel] = result_x

#构造最终的影响结果
result = pd.DataFrame(result_dic,index = ['value']).T

result

,value
A,-0.090909
B,0.000000
C,0.166667
D,-0.130435


#### 整合计算贡献率

In [24]:
result['贡献率'] = result['value'] / result['value'].sum()
result

,value,贡献率
A,-0.090909,1.662651
B,0.000000,-0.000000
C,0.166667,-3.048193
D,-0.130435,2.385542


# 14.2 Adtributor算法

## 14.2.1 Adtributor介绍

## 14.2.2 单个维度的基础案例

## 14.2.3 多个维度的算法逻辑和Pandas实现

#### 构造案例数据

In [15]:
#单位都是万

data = pd.DataFrame({'维度':['渠道','渠道','渠道','渠道','新老客','新老客','产品','产品','产品','产品','产品','产品'],
                     '元素':['A','B','C','D','新客','老客','C001','C002','C003','C004','C005','C006'],
                     'before':[50,3,18,4,35,40,20,15,10,8,12,10],
                     'after':[78,23,45,4,60,90,40,38,15,20,17,20]})
data

,维度,元素,before,after
0,渠道,A,50,78
1,渠道,B,3,23
2,渠道,C,18,45
3,渠道,D,4,4
4,新老客,新客,35,60
5,新老客,老客,40,90
6,产品,C001,20,40
7,产品,C002,15,38
8,产品,C003,10,15
9,产品,C004,8,20


### 1.计算惊讶度并排序

#### 计算出每个维度下各元素的p、q的值

In [16]:
#计算活动前销售额汇总和活动后销售额汇总
#因为有3个大维度，直接汇总是3倍销售额，需要除以维度数量，这里len(data['维度'].unique()等于3
pre_sum = data['before'].sum() / len(data['维度'].unique()) 
aft_sum = data['after'].sum() / len(data['维度'].unique()) 

#计算p和q值
data['p'] = data['before'] / pre_sum
data['q'] = data['after'] / aft_sum

In [17]:
data

,维度,元素,before,after,p,q
0,渠道,A,50,78,0.666667,0.520000
1,渠道,B,3,23,0.040000,0.153333
2,渠道,C,18,45,0.240000,0.300000
3,渠道,D,4,4,0.053333,0.026667
4,新老客,新客,35,60,0.466667,0.400000
5,新老客,老客,40,90,0.533333,0.600000
6,产品,C001,20,40,0.266667,0.266667
7,产品,C002,15,38,0.200000,0.253333
8,产品,C003,10,15,0.133333,0.100000
9,产品,C004,8,20,0.106667,0.133333


#### 惊讶度S的计算

In [18]:
import math
#创建一个列表用于收集每个s值
surprises = []

#遍历计算每一行数据的s值
for p,q in zip(data['p'],data['q']):
    #用JS散度公式
    s = 0.5 * (p * math.log10(2 * p / (p + q)) + q * math.log10(2 * q / (p + q)))
    surprises.append(s)

#把计算好的s列表赋值给data
data['surprise'] = surprises

#每个维度下按照s值排序
data.sort_values(['维度','surprise'],ascending = False,inplace = True)

data

,维度,元素,before,after,p,q,surprise
1,渠道,B,3,23,0.040000,0.153333,0.007697
0,渠道,A,50,78,0.666667,0.520000,0.001973
3,渠道,D,4,4,0.053333,0.026667,0.000984
2,渠道,C,18,45,0.240000,0.300000,0.000725
4,新老客,新客,35,60,0.466667,0.400000,0.000557
5,新老客,老客,40,90,0.533333,0.600000,0.000426
10,产品,C005,12,17,0.160000,0.113333,0.000869
7,产品,C002,15,38,0.200000,0.253333,0.000683
8,产品,C003,10,15,0.133333,0.100000,0.000519
9,产品,C004,8,20,0.106667,0.133333,0.000322


### 2.计算EP值并筛选元素

#### 计算每一行数据的EP值

In [20]:
#计算出总销售波动，3个维度都在一起，因此也需要除以维度数量
sum_dif = (data['after'].sum() - data['before'].sum()) / len(data['维度'].unique()) 

#计算每一行数据的EP
data['EP'] = (data['after'] - data['before']) / sum_dif

data.head(6)

,维度,元素,before,after,p,q,surprise,EP
1,渠道,B,3,23,0.040000,0.153333,0.007697,0.266667
0,渠道,A,50,78,0.666667,0.520000,0.001973,0.373333
3,渠道,D,4,4,0.053333,0.026667,0.000984,0.000000
2,渠道,C,18,45,0.240000,0.300000,0.000725,0.360000
4,新老客,新客,35,60,0.466667,0.400000,0.000557,0.333333
5,新老客,老客,40,90,0.533333,0.600000,0.000426,0.666667


#### 设定单个元素EP阈值teep和总EP阈值tep，这里分别将teep和tep设置为0.2与0.8

- 根据设定的单个元素EP阈值teep，遍历所有元素的EP值是否高于0.2，如果高于，则通过筛选
- 每一次遍历的同时，把每个维度下通过筛选的元素EP值累加，如果某维度下累加的EP值大于0.8，则该维度停止筛选

In [21]:
#假设单个EP阈值teep = 0.2，总EP阈值tep = 0.8
teep = 0.2
tep = 0.8

#筛选出EP值大于单个EP阈值teep的元素
data_fil = data.loc[data['EP'] >= teep,['维度','元素','surprise','EP']]

#新建一个EP_sum列，即每个维度内EP值做累加，作为和总阈值tep对比的辅助列
data_fil['EP_sum'] = data_fil.groupby('维度')['EP'].cumsum()
data_fil

,维度,元素,surprise,EP,EP_sum
1,渠道,B,0.007697,0.266667,0.266667
0,渠道,A,0.001973,0.373333,0.640000
2,渠道,C,0.000725,0.360000,1.000000
4,新老客,新客,0.000557,0.333333,0.333333
5,新老客,老客,0.000426,0.666667,1.000000
7,产品,C002,0.000683,0.306667,0.306667
6,产品,C001,0.000000,0.266667,0.573333


#### 根据总阈值进行筛选

In [22]:
#先筛选出大于总阈值的数据
bri = data_fil.loc[data_fil['EP_sum'] >= tep,:]

#每个维度下，把超过总阈值的第一个累加EP值作为接下来的筛选门槛
bri_dim = bri.groupby('维度').head(1)[['维度','EP_sum']]

#把经过单个阈值teep筛选后的数据和每个维度筛选门槛相匹配，用于下一步计算，空缺值用总阈值tep填充
result = pd.merge(data_fil,bri_dim,left_on = '维度',right_on = '维度',how = 'left').fillna(tep)
result.columns = ['维度','元素','S','EP','EP_sum','EP_thres']

#剔除大于筛选门槛的数据，即筛选出小于或等于筛选门槛的数据
result = result.loc[result['EP_sum'] <= result['EP_thres'],:]

result

,维度,元素,S,EP,EP_sum,EP_thres
0,渠道,B,0.007697,0.266667,0.266667,1.0
1,渠道,A,0.001973,0.373333,0.640000,1.0
2,渠道,C,0.000725,0.360000,1.000000,1.0
3,新老客,新客,0.000557,0.333333,0.333333,1.0
4,新老客,老客,0.000426,0.666667,1.000000,1.0
5,产品,C002,0.000683,0.306667,0.306667,0.8
6,产品,C001,0.000000,0.266667,0.573333,0.8


### 3.返回最终结果

In [23]:
result_gp = result.groupby('维度')['S'].sum().reset_index()
print(result_gp)

    维度         S
0   产品  0.000683
1  新老客  0.000983
2   渠道  0.010395


In [24]:
#筛选出前n个影响最大的维度
n = 1

#每个维度按照surprise排序，并返回前n个维度
top_n = result_gp.sort_values('S',ascending = False).iloc[:n,:]

#根据选择的前n个维度，返回维度对应的元素具体数据
result.loc[result['维度'].isin(top_n['维度']),:]

,维度,元素,S,EP,EP_sum,EP_thres
0,渠道,B,0.007697,0.266667,0.266667,1.0
1,渠道,A,0.001973,0.373333,0.640000,1.0
2,渠道,C,0.000725,0.360000,1.000000,1.0
